In [ ]:
pip install numpy

: 

In [ ]:
pip install pandas 

In [ ]:
pip install requests

In [ ]:
pip install bs4

In [15]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from datetime import datetime, timedelta
import requests
import numpy as np
import pandas as pd

In [ ]:
base_url = "http://envapp.maricopa.gov/EnvironmentalHealth/FoodGrade?"

In [ ]:
# Get the current date
today = datetime.now()

# If today is Saturday or Sunday, set days_to_friday to 0 to get the previous Friday
days_to_friday = (today.weekday() - 4) % 7

# Subtract the days to get the Friday date
last_friday = today - timedelta(days=days_to_friday)

# Format the date as "mm/dd/yyyy"
friday_date_str = last_friday.strftime("%m/%d/%Y")

# Construct the complete URL
full_url = f"{base_url}d={friday_date_str}&a=true"

# Open the URL and parse with BeautifulSoup
url = urlopen(full_url)
soup_doc = bs(url, "html.parser")
table = soup_doc.find_all("tr")

In [ ]:
master_data = []

for row in table[1:]: #skip the first row, it contains headers
  rest_list = row.find_all('td')
  # Get the link from the Permit ID td by searching for its class attribute
  inspection_link = row.find('td', {'class': 'boldTextCenter'}).a['href'] # grab the a tag's href attribute
  list_headers = ['Business Name', 'Address', 'City', 'Permit_ID', 'Type', 'Class', 'Inspection_Date', 'Inspection_Type', 'Grade', 'Priority_Violations', 'CuttingEdge','Inspection_Details'] # added link to the column heads
  rest_data = []
  for index, col in enumerate(rest_list):
    #print(index, col)
    rest_data.append(col.text)
  rest_data.append('http://envapp.maricopa.gov' + inspection_link) # appending the base url to the insepction subquery

  rest_dict = dict(zip(list_headers, rest_data))
  master_data.append(rest_dict)

master_data

[{'Business Name': 'Catalina Ventura Elementary School',
  'Address': '6331 N 39th Ave  ',
  'City': 'Phoenix',
  'Permit_ID': 'FD-00063',
  'Type': 'School Food Service    ',
  'Class': 'Class 3',
  'Inspection_Date': '03/04/2024',
  'Inspection_Type': 'Routine Inspection',
  'Grade': 'A',
  'Priority_Violations': '0',
  'CuttingEdge': '',
  'Inspection_Details': 'http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-00063&i=5662622'},
 {'Business Name': 'Hopi Elementary School',
  'Address': '5110 E Lafayette Blvd  ',
  'City': 'Scottsdale',
  'Permit_ID': 'FD-00113',
  'Type': 'School Food Service    ',
  'Class': 'Class 3',
  'Inspection_Date': '03/05/2024',
  'Inspection_Type': 'Routine Inspection',
  'Grade': 'A',
  'Priority_Violations': '0',
  'CuttingEdge': '',
  'Inspection_Details': 'http://envapp.maricopa.gov/EnvironmentalHealth/FoodInspection?p=FD-00113&i=5663605'},
 {'Business Name': 'Villa De Paz Elementary School',
  'Address': '4940 N 103rd Ave  ',
  'City

In [ ]:
pd.option_context('display.max_colwidth', None)
df = pd.DataFrame(master_data)
df.shape

(596, 12)

In [ ]:
df = df[['Business Name', 'Address', 'City', 'Permit_ID', 'Type', 'Class', 'Inspection_Date', 'Inspection_Type', 'Grade', 'Priority_Violations', 'Inspection_Details']]

In [ ]:
df.to_csv('data/databaseupdate.csv')